1. You have been given a partially implemented code for a feed-forward neural network using PyTorch. Your task is to complete the missing parts of the code to make it functional.

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the neural network architecture
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)# Complete this line to pass the output of the first layer through the activation function
        x = self.relu(x)# Complete this line to pass the output of the activation function through the second layer
        return x

# Define the hyperparameters
input_size = 10
hidden_size = 20
label_size = 5
learning_rate = 0.001
num_epochs = 1000

# Create the neural network object
model = NeuralNetwork(input_size, hidden_size, label_size)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Generate some dummy data for training
train_data = torch.randn(100, input_size)
train_labels = torch.randint(label_size, (100,))

# Training loop
for epoch in range(num_epochs):
    # Forward pass
    model.train(True)
    # Complete this line to pass the training data through the model and obtain the predictions
    outputs = model(train_data)

    # Compute the loss
    loss = criterion(outputs, train_labels)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch: {epoch+1}/{num_epochs}, Loss: {loss.item()}')

# Test the trained model
test_data = torch.randn(10, input_size)
with torch.no_grad():
    # Complete this line to pass the test data through the model and obtain the predictions
    test_outputs = model(test_data)

    # Print the predictions
    _, predicted = torch.max(test_outputs.data, 1)
    print("Predictions:", predicted)

Epoch: 100/1000, Loss: 1.609714150428772
Epoch: 200/1000, Loss: 1.608396053314209
Epoch: 300/1000, Loss: 1.6071648597717285
Epoch: 400/1000, Loss: 1.605959415435791
Epoch: 500/1000, Loss: 1.6047974824905396
Epoch: 600/1000, Loss: 1.6036269664764404
Epoch: 700/1000, Loss: 1.602494478225708
Epoch: 800/1000, Loss: 1.6013919115066528
Epoch: 900/1000, Loss: 1.600313425064087
Epoch: 1000/1000, Loss: 1.5991991758346558
Predictions: tensor([4, 2, 0, 2, 4, 2, 4, 2, 2, 0])


2. In this coding exercise, you need to implement the training of a deep MLP on the MNIST dataset using PyTorch and manually tune the hyperparameters. Follow the steps below to proceed:

* Load the MNIST dataset using torchvision.datasets.MNIST. The dataset contains handwritten digit images, and it can be easily accessed through PyTorch's torchvision module.

In [6]:
import os
os.getcwd()

'd:\\programming\\deeplearning\\exercise\\W2'

In [7]:
# Load the MNIST dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
train_mnist=datasets.MNIST(root='./mnist_data',train=True,download=True,transform=ToTensor())
test_mnist=datasets.MNIST(root='./mnist_data',train=False,transform=ToTensor())

100%|██████████| 9912422/9912422 [00:26<00:00, 368718.52it/s]


Extracting ./mnist_data\MNIST\raw\train-images-idx3-ubyte.gz to ./mnist_data\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 683386.33it/s]


Extracting ./mnist_data\MNIST\raw\train-labels-idx1-ubyte.gz to ./mnist_data\MNIST\raw



100%|██████████| 1648877/1648877 [00:03<00:00, 412567.06it/s]


Extracting ./mnist_data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./mnist_data\MNIST\raw



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting ./mnist_data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./mnist_data\MNIST\raw



* Define your deep MLP model. Specify the number of hidden layers, the number of neurons in each layer, and the activation function to be used. You can use the nn.Sequential container to stack the layers.

In [60]:
class MLP(nn.Module):
    def __init__(self,in_feature:int,output_feature:int,*args,**kwargs):
        super().__init__()
        # self.flatten=nn.Flatten()
        self.l1=nn.Linear(in_feature,100)
        self.l2=nn.Linear(100,50)
        self.l3=nn.Linear(50,output_feature)
    
    def forward(self, x):
        # x=self.flatten(x)
        x=self.l1(x)
        output=nn.ReLU()(x)
        output=self.l2(output)
        output=nn.ReLU()(output)
        return self.l3(output)

* Set up the training loop and the hyperparameters. You can use the CrossEntropyLoss as the loss function and the Stochastic Gradient Descent (SGD) optimizer.

In [61]:
from torch.utils.data import DataLoader
# Set hyperparameters
learning_rate = 0.1
epochs = 10
batch_size = 64

# Create data loaders
train_loader = DataLoader(train_mnist,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(test_mnist,batch_size=batch_size,shuffle=True)

# Create an instance of the model
model = MLP(784,10)

# Define the loss function and optimizer
criterion =  nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=learning_rate)

In [66]:
import tqdm

* Train the model by iterating over the training dataset for the specified number of epochs. Compute the loss, perform backpropagation, and update the model's parameters. 

In [70]:
# Training loop
import tqdm
for epoch in range(epochs):
    print(f'epoch: {epoch}')
    model.train(True)
    for batch_in,(images, labels)  in enumerate(train_loader): 
#         # Flatten the images
        images = images.view(-1, 784)
        outputs=model(images)
        loss=criterion(outputs,labels)
        

#         # Zero the gradients
        optimizer.zero_grad()
        
#         # Backward pass and optimization
        loss.backward()

#         # Forward pass
        optimizer.step()
print(loss)



epoch: 0
epoch: 1
epoch: 2
epoch: 3
epoch: 4
epoch: 5
epoch: 6
epoch: 7
epoch: 8
epoch: 9
tensor(0.0016, grad_fn=<NllLossBackward0>)


In [71]:
print(loss.item())

0.0016163301188498735


* Evaluate the trained model on the test dataset and calculate the accuracy (Please take a moment to consider the code below!)

In [72]:
# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.view(-1, 784)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print("Accuracy: {:.2f}%".format(accuracy))

Accuracy: 97.98%


* Manually tune the hyperparameters, such as the learning rate, by experimenting with different values and observing the performance. You can also search for the optimal learning rate by using techniques like learning rate range test, where you gradually increase the learning rate and monitor the loss.

In [ ]:
# TODO

3. In this coding exercise, you'll have an opportunity to explore the behavior of a deep neural network trained on the CIFAR10 image dataset. Follow the steps below:

* a. Construct a deep neural network (DNN) using 20 hidden layers, each comprising 100 neurons. To facilitate this exploration, employ the Swish activation function for each layer. Utilize nn.ModuleList to manage the layers effectively.

* b. Load the CIFAR10 dataset for training your network. Utilize the appropriate function, such as torchvision.datasets.CIFAR10. The dataset consists of 60,000 color images, with dimensions of 32×32 pixels. It is divided into 50,000 training samples and 10,000 testing samples. With 10 classes in the dataset, ensure that your network has a softmax output layer comprising 10 neurons. When modifying the model's architecture or hyperparameters, conduct a search to identify an appropriate learning rate. Implement early stopping during training and employ the Nadam optimization algorithm.

* c. Experiment by adding batch normalization to your network. Compare the learning curves obtained with and without batch normalization. Analyze whether the model converges faster with batch normalization and observe any improvements in its performance. Additionally, assess the impact of batch normalization on training speed.

* d. As an additional experiment, substitute batch normalization with SELU (Scaled Exponential Linear Units). Make the necessary adjustments to ensure the network self-normalizes. This involves standardizing the input features, initializing the network's weights using LeCun normal initialization (nn.init.kaiming_normal_), and ensuring that the DNN consists solely of dense layers. Observe the effects of utilizing SELU activation and self-normalization on the network's training stability and performance.

In [105]:
# TODO
class Cifar(nn.Module):
    def __init__(self, input_size,output_size,hidden_layer_size,hidden_layer_amount) -> None:
        super().__init__()
        self.input_layer=nn.Linear(input_size,hidden_layer_size)
        self.hidden_layer=nn.ModuleList()
        self.output_layer=nn.Linear(hidden_layer_size,output_size)
        self.swish=nn.SiLU()
        self.softmax=nn.Softmax()
        self.flatten=nn.Flatten()
        for _ in range(hidden_layer_amount):
            self.hidden_layer.append(nn.Linear(hidden_layer_size,hidden_layer_size))

    def forward(self,x):
        x=self.flatten(x)
        output=self.input_layer(x)
        output=self.swish(output)
        for layer in self.hidden_layer:
            output=layer(output)
            output=self.swish(output)
        output=self.output_layer(output)
        output=self.softmax(output)
        return output

In [106]:
cifar_train=datasets.CIFAR10(root='./Cfar10_data',train=True,download=True,transform=ToTensor())
cifar_test=datasets.CIFAR10(root='./Cfar10_data',train=False,download=True,transform=ToTensor())


Files already downloaded and verified
Files already downloaded and verified


In [107]:
learning_rate = 1e-4
epochs = 10
batch_size = 100
input_size=32*32*3
output_size=10
train_dloader=DataLoader(cifar_train,batch_size=batch_size,shuffle=True)
test_dloader=DataLoader(cifar_test,batch_size=batch_size,shuffle=True)

# Create an instance of the model
model = Cifar(input_size=input_size,output_size=output_size,hidden_layer_size=100,hidden_layer_amount=20)

# Define the loss function and optimizer
criterion =  nn.CrossEntropyLoss()
optimizer = optim.NAdam(model.parameters(),lr=learning_rate)

In [109]:
for epoch in range(epochs):
    model.train()

    for batch_in,(data,target) in enumerate(train_dloader):
        images=data.view(data.size(0),-1)
        outputs=model(images)
        loss=criterion(outputs,target)
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

    model.eval()
    with torch.no_grad():
        for test_data,test_target in test_dloader:
            toutput=model(test_data)
            tloss=criterion(toutput,test_target)
    
    print(f'epoch: {epoch}| train_loss: {loss.item()} | test_loss: {tloss.item()}')

d:\programming\deeplearning\Deeplearning\lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


epoch: 0| train_loss: 2.302555799484253 | test_loss: 2.3031837940216064
epoch: 1| train_loss: 2.3027424812316895 | test_loss: 2.3033342361450195
epoch: 2| train_loss: 2.3028271198272705 | test_loss: 2.3028364181518555
epoch: 3| train_loss: 2.30206561088562 | test_loss: 2.3021602630615234
epoch: 4| train_loss: 2.3027660846710205 | test_loss: 2.301896095275879
epoch: 5| train_loss: 2.302676200866699 | test_loss: 2.3021483421325684
epoch: 6| train_loss: 2.3027963638305664 | test_loss: 2.302309036254883
epoch: 7| train_loss: 2.302724838256836 | test_loss: 2.302990674972534
epoch: 8| train_loss: 2.3023667335510254 | test_loss: 2.302685022354126
epoch: 9| train_loss: 2.302807569503784 | test_loss: 2.302295446395874
